<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%204/Assignment_part_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DSML investigation:

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part four
### Due 22.10
#### Identifying the suspects credit score
We received informations that the rogue agent has a good credit score.

Our spies at SIMF have managed to collect financial information relating to our suspects as well as a training dataset.

Create a Neural Network over the training dataset `df` to identify which of the suspects have a good Credit_Mix


## Getting to know our data

* Age: a users age
* Occupation: a users employment field
* Annual_Income: a users annual income
* Monthly_Inh_Salary: the calculated salary received by a given user on a monthly basis
* Num_Bank_Accounts: the number of bank accounts possessed by a given user
* Num_Credit_Cards: the number of credit card given user possesses
* Interest_Rate: The interest rate on those cards (if multiple then its the average)
* Num_of_Loans: The number of loans of each user
* Delay_from_due_date: payment tardiness of user
* Num_of_Delayed_Payment: the count of delayed payments
* Changed_Credit_Limit: NaN
* Num_Credit_Inquiries: NaN
* Credit_Mix: The users credit score
* Outsting_Debt: Outstanding debt
* Credit_Utilization_Ratio: the percentage of borrowed money over borrowing allowance
* Payment_of_Min_Amount: does the user usually pay the minimal amount (categorical)
* Total_EMI_per_month: Monthly repayments to be made
* Amount_invested_monthly: The amout put in an investment fun by the user on a monthly basis
* Payment_Behaviour: the users payment behavior (categorical)
* Monthly_Balance: The users end of the month balance
* AutoLoan: If the user has an active loan for their vehicule
* Credit-BuilderLoan: If the user has a loan to increase their credit score
* DebtConsolidationLoan, HomeEquityLoan, MortgageLoan, NotSpecified, PaydayLoan, PersonalLoan, StudentLoan: different types of loans(categorical features)



In [61]:
# Import required packages
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [62]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/train_classification.csv", index_col='Unnamed: 0').dropna()
suspects = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/suspects.csv", index_col='Unnamed: 0').dropna()

In [63]:
df.head()

,Age,Occupation,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Monthly_Balance,AutoLoan,Credit-BuilderLoan,DebtConsolidationLoan,HomeEquityLoan,MortgageLoan,NotSpecified,PaydayLoan,PersonalLoan,StudentLoan
0,23,Scientist,19114.12,1824.843333,3,4,3,4,3,7,...,186.266702,1,1,0,1,0,0,0,1,0
1,24,Scientist,19114.12,1824.843333,3,4,3,4,3,9,...,361.444004,1,1,0,1,0,0,0,1,0
3,24,Scientist,19114.12,4182.004291,3,4,3,4,4,5,...,343.826873,1,1,0,1,0,0,0,1,0
5,28,Teacher,34847.84,3037.986667,2,4,6,1,3,3,...,303.355083,0,1,0,0,0,0,0,0,0
8,35,Engineer,143162.64,4182.004291,1,5,8,3,8,1942,...,854.226027,2,0,0,0,0,1,0,0,0


In [64]:
df["Credit_Mix"].value_counts()

Standard    13421
Good         8963
Bad          6839
Name: Credit_Mix, dtype: int64

# 1. Preparing the data
## 1.1 Data cleaning
 Perform OHE over the "Occupation" feature

 Then, perform LE over Payment_of_Min_Amount and Payment_Behaviour

 _hint: As we will be testing only one model no need to define a pipeline_

In [65]:
occupation_dummies = pd.get_dummies(df['Occupation'], prefix='Occupation')
df = pd.concat([df, occupation_dummies], axis=1)
df.drop('Occupation', axis=1, inplace=True)

In [66]:
le = LabelEncoder()

# Label Encoding for 'Payment_of_Min_Amount'
df['Payment_of_Min_Amount'] = le.fit_transform(df['Payment_of_Min_Amount'])

# Label Encoding for 'Payment_Behaviour'
df['Payment_Behaviour'] = le.fit_transform(df['Payment_Behaviour'])

## 1.2 Dataset splitting

Split the dataset in two, first X with your independent features and then y with the dependent feature **CreditMix**.

Then perform :
* OneHotEncoding over the **CreditMix** feature.
* A MinMaxScaller over the independent features

In [67]:
# Your code here:
X = df.drop('Credit_Mix', axis=1)
y = df['Credit_Mix']



#Define the scaler
y_dummies = pd.get_dummies(y, prefix='Credit_Mix')
# Define the scaler
scaler = MinMaxScaler()

# Fit and transform the scaler on independent features
X_scaled = scaler.fit_transform(X)

# Convert the scaled data back to a dataframe
X = pd.DataFrame(X_scaled, columns=X.columns)


### 1.2.1 Train Test splitting
Now split the data in X_train, X_test, y_train, y_test,

You can use test_size = 0.2 and a random_state of 42

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y_dummies, test_size=0.2, random_state=42)


### 1.2.2 final touches
Convert your datasets to `Torch tensors` of type `torch.float`

In [69]:
i#Your code here:
# Convert training data
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float)

# Convert testing data
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float)
print(X_train_tensor.size(), y_train_tensor.size())


torch.Size([23378, 42]) torch.Size([23378, 3])


# 2 Model preparation:

## 2.1 Define a Neural network model and instantiate it.
You can set the number of neurons to 150.

In [70]:
# Define a neural network class here:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        # Define layers
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.layer2(x)  # No activation function here since we're assuming a classification task.
        return x

# Define input, hidden, and output sizes
input_size = X_train.shape[1]
hidden_size = 150
output_size = y_train.shape[1]

In [71]:
# Instantiate your model here
model = NeuralNetwork(input_size, hidden_size, output_size)
print(model)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

NeuralNetwork(
  (layer1): Linear(in_features=42, out_features=150, bias=True)
  (layer2): Linear(in_features=150, out_features=3, bias=True)
  (relu): ReLU()
)
6903


## 2.2 finding the best model:
Identify, amongst the following options the best parameters for your model:

* `criterion` : [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), [BCEWithLogitsLoss](hhttps://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)
* `iterations` : 150, 250, 500
* `learning rate` : 0.00005, 0.001, 12.031


_Hint: restart your runtime between each execution to ensure that previous neural networks dont interfere with your current one_

_You can evaluate your model based on it's accuracy over the test set_

In [73]:
# Placeholder for best parameters and their accuracy
best_params = {
    "criterion": None,
    "iterations": None,
    "learning_rate": None,
    "accuracy": 0
}

# Potential hyperparameters to search over
criteria = [nn.CrossEntropyLoss, nn.BCEWithLogitsLoss]
iterations_list = [150, 250, 500]
learning_rates = [0.00005, 0.001]

for criterion in criteria:
    for iters in iterations_list:
        for lr in learning_rates:
            # Instantiate the model
            model = NeuralNetwork(input_size, hidden_size, output_size)
            loss_function = criterion()

            # Define Adam optimizer
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)

            # Training loop
            for epoch in range(iters):
                optimizer.zero_grad()
                outputs = model(X_train_tensor)

                if criterion == nn.CrossEntropyLoss:
                    loss = loss_function(outputs, torch.max(y_train_tensor, 1)[1])
                else:
                    loss = loss_function(outputs, y_train_tensor)

                loss.backward()
                optimizer.step()

            # Evaluate accuracy
            with torch.no_grad():
                predicted = model(X_test_tensor)
                if criterion == nn.CrossEntropyLoss:
                    _, predicted_labels = torch.max(predicted, 1)
                    _, true_labels = torch.max(y_test_tensor, 1)
                    accuracy = (predicted_labels == true_labels).float().mean()
                else:
                    accuracy = ((predicted > 0.5) == y_test_tensor).float().mean()

            # Print performance for the current combination
            print(f"Criterion: {criterion.__name__}, Iterations: {iters}, Learning Rate: {lr}, Accuracy: {accuracy:.4f}")

            if accuracy > best_params["accuracy"]:
                best_params["criterion"] = criterion
                best_params["iterations"] = iters
                best_params["learning_rate"] = lr
                best_params["accuracy"] = accuracy

# Print the best parameters after all combinations
print("\nBest Parameters:")
print(best_params)

Criterion: CrossEntropyLoss, Iterations: 150, Learning Rate: 5e-05, Accuracy: 0.4546
Criterion: CrossEntropyLoss, Iterations: 150, Learning Rate: 0.001, Accuracy: 0.7755
Criterion: CrossEntropyLoss, Iterations: 250, Learning Rate: 5e-05, Accuracy: 0.4556
Criterion: CrossEntropyLoss, Iterations: 250, Learning Rate: 0.001, Accuracy: 0.8068
Criterion: CrossEntropyLoss, Iterations: 500, Learning Rate: 5e-05, Accuracy: 0.5117
Criterion: CrossEntropyLoss, Iterations: 500, Learning Rate: 0.001, Accuracy: 0.8393
Criterion: BCEWithLogitsLoss, Iterations: 150, Learning Rate: 5e-05, Accuracy: 0.6667
Criterion: BCEWithLogitsLoss, Iterations: 150, Learning Rate: 0.001, Accuracy: 0.8098
Criterion: BCEWithLogitsLoss, Iterations: 250, Learning Rate: 5e-05, Accuracy: 0.6667
Criterion: BCEWithLogitsLoss, Iterations: 250, Learning Rate: 0.001, Accuracy: 0.8429
Criterion: BCEWithLogitsLoss, Iterations: 500, Learning Rate: 5e-05, Accuracy: 0.6667
Criterion: BCEWithLogitsLoss, Iterations: 500, Learning Rate

In [74]:
# Perform your iterations here

## 2.3 Model Accuracy
Identify the models accuracy over the train and test parts of the training dataset

In [75]:
# Instantiate the best model based on the parameters
best_model = NeuralNetwork(input_size, hidden_size, output_size)
best_criterion = nn.BCEWithLogitsLoss()
best_optimizer = torch.optim.Adam(best_model.parameters(), lr=0.001)

# Train the best model
for epoch in range(500):  # 500 iterations
    best_optimizer.zero_grad()
    outputs = best_model(X_train_tensor)
    loss = best_criterion(outputs, y_train_tensor)
    loss.backward()
    best_optimizer.step()

# Function to calculate accuracy
def calculate_accuracy(model, data, labels, criterion):
    with torch.no_grad():
        outputs = model(data)
        if criterion == nn.CrossEntropyLoss:
            _, predicted = torch.max(outputs, 1)
            _, true_labels = torch.max(labels, 1)
            accuracy = (predicted == true_labels).float().mean()
        else:  # BCEWithLogitsLoss
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            accuracy = (predicted == labels).float().mean()
    return accuracy.item()

# Calculate training accuracy
train_accuracy = calculate_accuracy(best_model, X_train_tensor, y_train_tensor, nn.BCEWithLogitsLoss)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

# Calculate test accuracy
test_accuracy = calculate_accuracy(best_model, X_test_tensor, y_test_tensor, nn.BCEWithLogitsLoss)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Training Accuracy: 88.69%
Test Accuracy: 88.13%


# 3. Predictions over the suspects dataset
## 3.1 Retrain a new model over the full training dataset
#### Please use the following parameters for this section:
* ``neurons`` = 150
* ``learning`` rate = 0.00005
* ``criterion`` = CrossEntropyLoss
* `iterations` = 500

_hint you may have to redo some preprocessing as you did in part one_

In [96]:
# Define a new model here:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 150)
        self.layer2 = nn.Linear(150, output_size)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        return x

model = NeuralNetwork()
# Define your MSE loss here:
criterion = nn.CrossEntropyLoss()
# Define your Adam optimizer for finding the weights of the network here:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)

# perform your training here
for epoch in range(500):

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/500], Loss: {loss.item()}')


Epoch [1/500], Loss: 1.1010977029800415
Epoch [2/500], Loss: 1.1007187366485596
Epoch [3/500], Loss: 1.1003406047821045
Epoch [4/500], Loss: 1.099963665008545
Epoch [5/500], Loss: 1.0995874404907227
Epoch [6/500], Loss: 1.099212408065796
Epoch [7/500], Loss: 1.098838448524475
Epoch [8/500], Loss: 1.0984654426574707
Epoch [9/500], Loss: 1.0980935096740723
Epoch [10/500], Loss: 1.0977226495742798
Epoch [11/500], Loss: 1.0973528623580933
Epoch [12/500], Loss: 1.0969840288162231
Epoch [13/500], Loss: 1.096616506576538
Epoch [14/500], Loss: 1.096250057220459
Epoch [15/500], Loss: 1.0958845615386963
Epoch [16/500], Loss: 1.0955201387405396
Epoch [17/500], Loss: 1.0951569080352783
Epoch [18/500], Loss: 1.0947948694229126
Epoch [19/500], Loss: 1.0944339036941528
Epoch [20/500], Loss: 1.0940741300582886
Epoch [21/500], Loss: 1.0937151908874512
Epoch [22/500], Loss: 1.0933575630187988
Epoch [23/500], Loss: 1.093000888824463
Epoch [24/500], Loss: 1.092645525932312
Epoch [25/500], Loss: 1.09229099

In [80]:
# perform your training here

## 3.2 Predict over the suspects dataset

In [97]:
# Predict which users have a good credit score here:
suspect_ids = suspects['userID']
suspects_for_prediction = suspects.drop(columns=['userID'])
suspects_tensor = torch.FloatTensor(suspects_for_prediction.values)

model.eval()

with torch.no_grad():
    outputs = model(suspects_tensor)
    _, predicted = torch.max(outputs, 1)
# Convert predictions to a list or array
predicted_list = predicted.numpy()

# Create a DataFrame to map 'suspectID' to predictions
results_df = pd.DataFrame({
    'suspectID': suspect_ids,
    'Prediction': predicted_list
})

# Identify which users have a good credit score
good_credit_score_users = results_df[results_df['Prediction'] == 1]['suspectID'].tolist()
good_credit_score_users

[317991,
 241892,
 303376,
 761992,
 373318,
 938770,
 676003,
 255073,
 468560,
 263592,
 660506,
 244162,
 774730,
 911014,
 601633,
 645054,
 255830,
 353527,
 531937,
 413250,
 601958,
 596063,
 764821,
 880689,
 340001,
 512604,
 165305,
 334769,
 564061,
 306495,
 647912,
 946059,
 917315,
 504629,
 520229,
 861915,
 308845,
 654987,
 398531,
 327507,
 888670,
 449514,
 819045,
 899768,
 251233,
 344946,
 932285,
 169664,
 265615,
 867078,
 618568,
 512128,
 222875,
 160212,
 466624,
 903438,
 957997,
 362662,
 892546,
 777470,
 397912,
 123562,
 939524,
 387676,
 860681,
 789726,
 167822,
 571370,
 863210,
 717780,
 863592,
 898254,
 424603,
 632973,
 599462,
 601265,
 402663,
 226609,
 790344,
 862416,
 134514,
 177842,
 551506,
 365406,
 825733,
 220420,
 927140,
 596661,
 688784,
 829946,
 242912,
 443524,
 464891,
 380835,
 823355,
 456546,
 790046,
 932281,
 538431,
 297653,
 206269,
 96336,
 273453,
 112768,
 446376,
 205381,
 614048,
 931353,
 773737,
 793505,
 419563,
 2